In [1]:
import torch
import torchvision
import numpy as np
!pip install onnx onnxruntime
import onnx
import onnxruntime
from PIL import Image, ImageDraw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00


In [2]:
# Установка пакета из форка репозитория onnx2torch
#!pip uninstall onnx2torch
#!pip install onnx2torch
!pip install -e git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch

Obtaining onnx2torch from git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch
  Cloning https://github.com/KaizerloveRussia/onnx2torch.git to ./src/onnx2torch
  Running command git clone --filter=blob:none --quiet https://github.com/KaizerloveRussia/onnx2torch.git /content/src/onnx2torch
  Resolved https://github.com/KaizerloveRussia/onnx2torch.git to commit ad46e9224f77ccaa06e5a327bc7269aac0778aba
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_c

In [6]:
# Загрузка и предобработка изображения
from google.colab import files
uploaded=files.upload()
image = Image.open("image.png").convert('RGB')
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
image = transform(image)
image = image.unsqueeze(0)

Saving image.png to image (2).png


In [7]:
# Загрузка и запуск предобученной модели Mask R-CNN
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()
output = model(image)
print(output)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[{'boxes': tensor([[202.2276,  38.5661, 414.1723, 628.7805],
        [ 20.9724,  63.0749, 205.0852, 640.0000],
        [483.5652, 339.3947, 540.7682, 382.8224],
        [270.5174,  48.3607, 329.9462, 183.4480],
        [159.5870, 118.8170, 223.4619, 485.4552],
        [ 10.0505,  74.4288, 224.0936, 375.2388],
        [233.5454,  45.9161, 429.4043, 383.6660],
        [ 35.7504, 263.9353, 178.8161, 634.7276],
        [458.5493, 415.2831, 588.8022, 487.1970],
        [150.5592, 166.9541, 205.6934, 360.8349],
        [264.4675,  43.4898, 341.5763, 241.8940],
        [158.1782, 176.0592, 205.4099, 419.4864],
        [190.9675, 138.7702, 237.8154, 385.6107],
        [484.7147, 358.7333, 521.4476, 383.9424],
        [265.7353,  48.1982, 335.3287, 240.4469],
        [296.2939,  98.0851, 415.1948, 553.2101]], grad_fn=<StackBackward0>), 'labels': tensor([ 1,  1,  3, 14,  1,  1,  1, 11, 15, 31,  1, 28, 28,  3, 11,  1]), 'scores': tensor([0.9988, 0.9953, 0.8488, 0.6591, 0.4875, 0.4059, 0.2914, 0.2

In [9]:
# Конвертация модели в ONNX
torch.onnx.export(model, image, "mask_rcnn.onnx", opset_version=13)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4009: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bo

In [10]:
# Загрузка и запуск модели в ONNX
onnx_model = onnx.load("mask_rcnn.onnx")
ort_session = onnxruntime.InferenceSession("mask_rcnn.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: image.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

[array([[202.22763 ,  38.566055, 414.17252 , 628.7805  ],
       [ 20.972258,  63.075016, 205.08545 , 640.      ],
       [483.56525 , 339.39474 , 540.7682  , 382.8224  ],
       [270.5175  ,  48.36055 , 329.9461  , 183.44731 ],
       [159.5868  , 118.816956, 223.46167 , 485.45547 ],
       [ 10.050376,  74.42878 , 224.09358 , 375.23874 ],
       [233.54462 ,  45.915775, 429.40448 , 383.66907 ],
       [ 35.750313, 263.93604 , 178.81566 , 634.72766 ],
       [458.54944 , 415.28314 , 588.8022  , 487.1971  ],
       [150.55905 , 166.95418 , 205.69345 , 360.83463 ],
       [264.4675  ,  43.48992 , 341.57642 , 241.89404 ],
       [158.17824 , 176.0592  , 205.41005 , 419.4868  ],
       [190.96759 , 138.7695  , 237.81549 , 385.61078 ],
       [484.71472 , 358.7335  , 521.4477  , 383.94232 ],
       [265.7353  ,  48.198433, 335.32864 , 240.44695 ],
       [296.29248 ,  98.084694, 415.19498 , 553.2109  ]], dtype=float32), array([ 1,  1,  3, 14,  1,  1,  1, 11, 15, 31,  1, 28, 28,  3, 11,  1]

In [11]:
# Обратная конвертация
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("mask_rcnn.onnx")

ModuleNotFoundError: No module named 'onnx2torch'